In [30]:
from urllib2 import Request, urlopen
from requests import Session
from lxml import html
import requests
import csv, sys
import pandas as pd

In [31]:
def getDefaultCommodityPrice ():
    '''
    Default scraper that fetch from two end point (gold and silver)
    Uses session and xpath to fetch data from the end point
    
    Output: Return void
        Two files in the ./ with gold.xlsx and silver.xlsx
    '''
    commodities = ['gold', 'silver']
    hist_substring = '-historical-data'
    file_extension = '.csv'
    csv_column = ['Date', 'Price', 'Open', 'High', 'Low', 'Vol', 'Change %']
    end_point = 'https://www.investing.com/commodities/'
    xpath_pattern = '//*[@id="curr_table"]/tbody/tr/td/text()'
    header = {'User-Agent': 'Mozilla/5.0'}

    for commodity in commodities:
        req = Request(''.join([end_point,commodity,hist_substring]), headers=header)
        webpage = urlopen(req).read()
        tree = html.fromstring(webpage)
        data = tree.xpath(xpath_pattern)
        filename = ''.join([commodity, file_extension])
        with open(filename, 'wb') as f:
            writer = csv.writer(f)
            writer.writerow(csv_column)
            for i in range(0, len(data), 7):
                writer.writerow(data[i:i+7])

In [32]:
getDefaultCommodityPrice()

In [33]:
def getCommodityPrice (start_date, end_date, commodity, interval='Daily'):
    '''
    This function will use a simple ajax post call to make session request
    to fetch raw commodity data in xml. 
    
    The CSV writer takes the commodity data from the parser and output
    
    Input:
        start_date: String
            Format: 03-03-2016 or 04/04/2016
            
        end_date: String
            Format:03-03-2016 or 04/04/2016
    
        commodity: String 
            Available options: {gold, silver}
   
    Output: tuple array of integer
            (mean, variance)
       
    Note: The output can be alter to avoid csv generation simply by keeping a running list
    
    Referene: 
        http://docs.python-requests.org/en/master/user/advanced/ #Usage of Session
        https://stackoverflow.com/questions/3030487/is-there-a-way-to-get-the-xpath-in-google-chrome #xpath
        https://docs.python.org/2/library/csv.html#writer-objects     #CSV writeable python
    
    '''
    default_referer = 'https://www.investing.com/commodities/'
    ajax_endpoint = 'https://www.investing.com/instruments/HistoricalDataAjax'
    xpath_pattern = '//*[@id="curr_table"]/tbody/tr/td/text()'
    user_agent = 'Mozilla/5.0'
    
    #Safe guard against invalid date 
    if len(start_date.split('-')) < 1 or len(end_date.split('-')) < 1:
        raise ValueError('Input date must in month-day-year format')
    
    st_date = '/'.join(start_date.split('-'))
    end_date = '/'.join(end_date.split('-'))
    
    file_extension = '.csv'
    csv_column = ['Date', 'Price', 'Open', 'High', 'Low', 'Vol', 'Change %']
    
    session = Session()
    
    #Safe guard against invalid commodity
    if commodity != 'gold'and commodity != 'silver':
        raise ValueError('Input commodity must be silver or gold')
        
    response = session.post(
    url=ajax_endpoint,
    data={
        'action': 'historical_data',
        'curr_id': '8830',
        'st_date': start_date,
        'end_date': end_date,
        'interval_sec': interval
    },
    headers={
        'Referer': ''.join([default_referer, commodity, '-historical-data']),
        'user-agent' : xpath_pattern,
        "X-Requested-With": "XMLHttpRequest"
    })
    
    tree = html.fromstring(response.text)
    data = tree.xpath(xpath_pattern)
    file_name = ''.join([commodity, file_extension])
    
    #CSV generation writing the default column and commodity price
    with open(file_name, 'wb') as f:
        writer = csv.writer(f)
        try:
            writer.writerow(csv_column)
            for i in range(0, len(data), 7):
                row = data[i:i+7]
                writer.writerow(data[i:i+7])
        except csv.Error as e:
            sys.exit('file %s, word %d: %s' % (filename, i, e))        
            
    data = pd.read_csv(file_name, delimiter=',')
    price = data.iloc[:,1].apply(lambda x: float(str(x).replace(',', '')))
    mean = price.mean()
    variance = price.var()
    return mean, variance

In [34]:
print getCommodityPrice('03/05/2017', '03/08/2017', 'gold')

(1216.0333333333333, 64.65333333333342)
